**MNIST BL model using ResNet**

In [1]:
import torch
import torch.nn as nn

from torchvision import datasets, transforms
from torchvision.models import resnet18
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim


In [2]:
# check device

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
# data preprocessing
# resize to 224*224

transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5),(0.5))
])

In [4]:
# load data
train_dataset = datasets.MNIST(
    root='./data', train=True, transform=transform, download=True
)
test_dataset = datasets.MNIST(
    root='./data', train=False, transform=transform, download=True
)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False, num_workers=2)

100%|██████████| 9.91M/9.91M [00:01<00:00, 5.13MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 58.3kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 1.26MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 6.70MB/s]


In [5]:
# load model
model = resnet18(pretrained = False)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [6]:
# modify the model
# conv1 greyscale
model.conv1 = nn.Conv2d(1,64, kernel_size=7, stride=2,padding=3, bias=False)
# altegether 10 classes
model.fc = nn.Linear(model.fc.in_features, 10)

In [7]:
#send to device
model = model.to(device)

In [8]:
#define loss function and optimizer
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)


In [9]:
# train

def train(model, device, train_loader, optimizer, epoch):
    print('Starting Training')
    model.train()

    for idx, (data, target) in enumerate(train_loader):

        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()

        outputs = model(data)  ###
        loss = loss_func(outputs, target)
        loss.backward()
        optimizer.step()   ###

        if idx % 100 == 0:
            print(f'Epoch {epoch}, [{idx * len(data)}/{len(train_loader.dataset)}] loss {loss.item():.3f}')

    print('Finished Training')

In [10]:
# test
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)

            test_loss += loss_func(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader)
    accuracy = 100. * correct / len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({accuracy:.2f}%)\n')

In [11]:
# train and test

for epoch in range(1,6):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

Starting Training
Epoch 1, [0/60000] loss 2.327
Epoch 1, [6400/60000] loss 0.080
Epoch 1, [12800/60000] loss 0.026
Epoch 1, [19200/60000] loss 0.065
Epoch 1, [25600/60000] loss 0.026
Epoch 1, [32000/60000] loss 0.115
Epoch 1, [38400/60000] loss 0.092
Epoch 1, [44800/60000] loss 0.028
Epoch 1, [51200/60000] loss 0.014
Epoch 1, [57600/60000] loss 0.009
Finished Training

Test set: Average loss: 0.0383, Accuracy: 9854/10000 (98.54%)

Starting Training
Epoch 2, [0/60000] loss 0.037
Epoch 2, [6400/60000] loss 0.165
Epoch 2, [12800/60000] loss 0.018
Epoch 2, [19200/60000] loss 0.036
Epoch 2, [25600/60000] loss 0.020
Epoch 2, [32000/60000] loss 0.002
Epoch 2, [38400/60000] loss 0.022
Epoch 2, [44800/60000] loss 0.013
Epoch 2, [51200/60000] loss 0.037
Epoch 2, [57600/60000] loss 0.020
Finished Training

Test set: Average loss: 0.0351, Accuracy: 9888/10000 (98.88%)

Starting Training
Epoch 3, [0/60000] loss 0.004
Epoch 3, [6400/60000] loss 0.263
Epoch 3, [12800/60000] loss 0.011
Epoch 3, [19200